# $$ \textbf{African Institute for Mathematical Sciences, Rwanda} $$

## $$ \textbf{Algebra and Cryptography Assignment 2} $$

### $$ \text{Chukwuezi Tochukwu Precious} $$

#### $$ \text{March 8, 2025} $$

# $\text{Useful Modules}$

In [49]:
#0 Sieve of Erastosthenes for finding the kth prime number in a range n = klnk
def sieve_of_eratosthenes(n, kth):
    prime = [True for i in range(n+1)]
    p = 2
    while (p * p <= n):
        if (prime[p] == True):
 
            # Update all multiples of p
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1
 
    # Print all prime numbers
    k = 0
    for p in range(2, n+1):
        if prime[p]:
            k += 1
        if k == kth:
            return p

#0.1 For finding the number of prime numbers between m and n, for m > n.      
def sieve_of_eratosthenes_between(m, n):
    if m < n:
        return "You must put the bigger number first."
    
    prime = [True for i in range(m+1)]
    p = 2
    while (p * p <= m):
        if (prime[p] == True):
 
            # Update all multiples of p
            for i in range(p * p, m+1, p):
                prime[i] = False
        p += 1
 
    # log prime numbers
    count_to_n = 0
    count_to_m = 0
    for p in range(2, m+1):
        if prime[p] and p <= m: #counts primes less or equal to m
            count_to_m += 1
        if prime[p] and p <= n: # countr primes less or equal to n
            count_to_n += 1
    
    d = count_to_m - count_to_n
    return {
        "difference" : d, 
        f"# of primes up to {m}" : count_to_m,
        f"# of primes up to {n}" :count_to_n
    }
    return f"There are {count_to_m} primes up to {m} and {count_to_n} primes up to {n}. So the primes between them are exactly {d}."

#1 Extended Euclidean Algorithm
def EEA(a, b):
    """My implementation uses r as the pivot"""
    # r = [max(a, b), min(a, b), max(a, b) % min(a, b)]
    r = [a, b, a % b]
    # r = [max(a, b), min(a, b)]
    u = [1, 0]
    v = [0, 1]
    q = [0, 0]
    k = 2
    while r[k]!= 0:
        
        # Add Quotiend and Remainder
        q.append(r[k-2] // r[k-1])

        #update v and u
        u.append(u[k-2] - u[k-1]*q[k])
        v.append(v[k-2] - v[k-1]*q[k])

        # Update r
        r.append(r[k-1]%r[k])

        # Increament
        k += 1
        
    if v[k-1] < 0:
        v[k-1] = (v[k-1] + a)%a

    
    return r[k-1], u[k-1], v[k-1]

#2. Generator Test Function
def is_generator(o_G, G, g):
        p_list = list(map(lambda x: x[0], factor(o_G)))
        test = all([not (power_mod(g, o_G//b, G) == 1) for b in p_list])
        return test
    
# is_generator(20, 25, 16)

#3. Order Test Function
def order_of(G):
    p_list = list(map(lambda x: x[0], factor(G)))
    prod = G
    for p in p_list:
        prod *= (1 - 1/p)
    return int(prod)

# order_of(210)

#4 Chinese Remainder Function
def chs_remainder(a, n, b, m):
    if not (EEA(n, m)[0] == 1):
        print("Invalid Inputs on m and n. They must be coprime")
        return
    else:
        _, u, v = EEA(n, m)
        S = b*u*n + a*v*m
        X = S % (m*n)
        return X, m*n
# chs_remainder(1, 6, 3, 5)

#5 Babysteps Giant Steps Algorithm    
def baby_steps_alg(g,x,d,G):
    "Application of Baby_Steps Algorithm Implemented in Class"
    
    m = ceil(sqrt(d))
    
    # Formation of the baby_steps
    baby_steps = [x]
    
    for idx in range(1, m+1):
        baby_steps.append(power_mod(baby_steps[idx-1]*g, 1, G))
    k1 = EEA(G, x)[2]
    last = baby_steps[-1]
    gm = power_mod(last*k1, 1, G)
    giant_steps = [1]
    i = 0 # index of collision
    j = 0
    
    for idx in range(1, m+1):
        if giant_steps[idx-1] in baby_steps:
            i = idx-1
            j = baby_steps.index(giant_steps[i])
            break
        giant_steps.append(power_mod(giant_steps[idx-1]*gm, 1, G))
        
        if giant_steps[idx] in baby_steps:
            i = idx
            j = baby_steps.index(giant_steps[i])
            break
    
    return baby_steps, giant_steps, i, j, power_mod(i*m - j, 1, d)

# baby_steps_alg(2, 17, 20, 25)

#4. Discrete Log Function (Trian and Error Version for Small p)
def trial_and_error(b, a, p, max_iter = 1000):
    
    x = 0
    counter = 0
    
    for i in range(0, max_iter+1):
        counter += 1
        if power_mod(b, i, p) == a:
            x = i
            return f"dlog_{b}({a}) = {x} in Z_{p}, after {counter} iterations"
    return "Time Out", 


#5 Discrete logs, Advanced (Pollig-Hellman)
def pollig_hellman_ones(g, x, p):
    from functools import reduce
    
    factor_list = list(factor(p-1))
    factor_primes = [f[0] for f in factor_list]
    factor_powers = [f[1] for f in factor_list]
    
    gi = []
    xi = []
    mi = []
    
    if all(e == 1 for e in factor_powers):
               
        for i in range(len(factor_list)):
            
            gi.append(power_mod(g, (p-1)//factor_primes[i], p))
            xi.append(power_mod(x, (p-1)//factor_primes[i], p))
            mi.append((baby_steps_alg(gi[i], xi[i], factor_primes[i], p)[-1], factor_primes[i]))
            
    
        # Implementing the Chinese Remainder
        a = list(reduce(lambda a, b : chs_remainder(a[0], a[1], b[0], b[1]), mi))

        return a


def pollig_hellman_singles(g, x, p, q=1):
    
    factor_list = list(factor(p-1))
    pf = q #a prime factor of p
    e = next(e for q1, e in factor_list if q1 == q)
    
    ai = []
    hi = []
    xi = [x]
    
    h = power_mod(g, pf^(e - 1), p)
    g_inverse = EEA(p, g)[2]
    
    for i in range(1, e+1):
        ai.append(baby_steps_alg(h, power_mod(xi[i-1], pf^(e - i), p), pf, p)[-1])
        xi.append(power_mod(xi[i-1]*g_inverse^(ai[i-1]*pf^(i-1)), 1, p))
    
    a = 0
    for j in range(e):
        a = a + power_mod(ai[j]*pf^j, 1, p)
    return a


def pollig_hellman_general(g, x, p):
    from functools import reduce
    """General Pollig-Hellman algorithm for any cyclic group of order p-1"""
    factor_list = list(factor(p-1))
    mi = []

    for q, e in factor_list:
        q_e = q^e
        g_i = power_mod(g, (p-1) // q_e, p)
        x_i = power_mod(x, (p-1) // q_e, p)
        dlog_i = pollig_hellman_singles(g_i, x_i, p, q)
        mi.append((dlog_i, q_e))

    # Reduce using Chinese Remainder Theorem
    return reduce(lambda a, b: chs_remainder(a[0], a[1], b[0], b[1]), mi)


# print(pollig_hellman_ones(2, 10, 211))
# print(pollig_hellman_singles(3, 7, 2^4 + 1, 2))  # if q is not a prime factor of p, there will be an error.
# print(pollig_hellman_general(7, 6, pt))


###########################################################################################################################
# MODULES FOR PRIMES 
###########################################################################################################################
import random

def is_primed(n, k=128):
    """
    Miller-Rabin primality test to check if n is prime.

    Args:
        n (int): The number to test.
        k (int): Number of iterations for accuracy.

    Returns:
        bool: True if prime, False otherwise.
    """
    if n in (2, 3):
        return True
    if n < 2 or n % 2 == 0:
        return False

    # Express (n - 1) as (2^s) * r, with r odd
    s, r = 0, n - 1
    while r % 2 == 0:
        s += 1
        r //= 2

    # Perform Miller-Rabin test
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, r, n)
        if x in (1, n - 1):
            continue

        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False  # Composite found

    return True  # Probably prime

def generate_prime(bits):
    """
    Generate a prime number of the given bit length.

    Args:
        bits (int): Number of bits.

    Returns:
        int: A prime number.
    """
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << bits - 1) | 1  # Ensure MSB and LSB are 1
        if is_primed(candidate, k=128):
            return candidate

def generate_special_prime(bits=1024):
    """
    Generate a prime number p = 2 * p1 * p2 + 1 where p1 and p2 are prime, 
    satisfying p1 < p2 < p1^3.

    Args:
        bits (int): Bit length of the target prime p.

    Returns:
        (int, int, int): The prime p and its prime factors (p1, p2).
    """
    while True:
        p1 = generate_prime(bits // 4)  # Smaller prime factor
        upper_bound = min(p1**3, 2**(bits // 2))  # Ensure p2 < p1^3
        while True:
            p2 = generate_prime(random.randint(bits // 4, bits // 2))
            if p1 < p2 < upper_bound:
                break

        p = 2 * p1 * p2 + 1
        if is_primed(p, k=128):
            return p, p1, p2  # Return prime and its factors

# $\textbf{Solutions 1.4}$

Use the Eratosthenes sieve to find the $(5000000 + n)$-th prime number where $n$ is your birthday (mmddyy). How many prime numbers do we have between $2^{25}$ and $2^{26}$?

### $1.4a \quad \textbf{The $(5000000 + n)^{th}$ Prime Number}$

Using the formula

$$
\pi(x) = \frac{x}{\ln{x}}
$$

Where $\pi(x)$ is approximately the number of prime numbers up to $x$ when $x$ is large.

And the formula

$$
R = \lceil N\ln{N}\rceil
$$

$R$ denotes the search range wide enough to find the $N^{th}$ prime number. So my birthday is $n = 30031997$. Using this, I evaluate 

$$
C = 5000000 + n = 5000000 + 30031997 = 35031997
$$

Therefore $R = \lceil35031997 \ln(35031997)\rceil > 608567879$

The code is implemented below, using the ```sieve_of_eratosthenes``` function.

In [2]:
%%time
C = 35031997
R = 700000000 # I made this number very large compared to the calculated value 
b = sieve_of_eratosthenes(R, C)# Find the nth prime number in the range N, takes about 2 to 5 minutes
b

CPU times: user 6min, sys: 10.7 s, total: 6min 11s
Wall time: 6min 10s


675156929

The Output shows that the $35031997^{th}$ prime number is $675156929$

In [13]:
nth = 675156929
is_safe_prime(675156929)

True

### $1.4b \quad \textbf{The number of Prime Numbers between $2^{25}$ and $2^{26}$}$

Using the $pi(x)$ function, we can approximate the number of prime numbers between $2^{25}$ and $2^{26}$ as

$$
\pi(2^{26}) - \pi(2^{25}) \approx \frac{2^{26}}{\ln(2^{26})}- \frac{2^{25}}{\ln(2^{25})} \approx 1787402 \approx 1.8 million.
$$

Implementing the code, we use the ```sieve_of_eratosthenes_between``` function as follows:

In [15]:
%%time
m = 2^26
n = 2^25
sieve_of_eratosthenes_between(m, n)

CPU times: user 42.8 s, sys: 1.12 s, total: 44 s
Wall time: 44 s


{'difference': 1894120,
 '# of primes up to 67108864': 3957809,
 '# of primes up to 33554432': 2063689}

Our output gives that there are $1894120$ prime numbers between $2^{25}$ and $2^{26}$, in the order of $1.8$ million like we estimated above.

# $\textbf{Solutions 2}$

Bob intercepts from Alice the following encrypted message:

$$
[[83025882561049910713, 66740266984208729661],
[117087132399404660932, 44242256035307267278],
[67508282043396028407, 77559274822593376192],
[60938739831689454113, 14528504156719159785],
[5059840044561914427, 59498668430421643612],
[92232942954165956522, 105988641027327945219],
[97102226574752360229, 46166643538418294423]]
$$

To communicate privately, Alice and Bob are using Elgamal cryptosystem. They have
choosen the cyclique group $F_p$ where

$p = 123456789987654353003$
and generated by $g = 123456789$

Their public keys are the following respectively:

$g_A = 52808579942366933355$, $g_B = 39318628345168608817$

Knowing that Alice and Bob agreed that: Each message consists of a single letter which is
encoded as:

$A = 11$, $B = 12$, $\cdots$ ,$Z = 36$ space $= 41$, $' = 42$ $ . = 43$, $, = 44$ $? = 45$

can you help Bob to decrypt the message?

### $2.0 \quad \textbf{Stepwise Approach}$

1. Input $(y, A)$ for each block, noting the messages are of the form $(A, y)$.
2. Compute $k = A^b$ $\bmod{p} \in G$, where $K = B^a = (g^b)^a = (g^a)^b = A^b$
3. Use $EEA$ to get $k^{-1}$
4. Get $m = y \cdot k^{-1} \in G$

In [3]:
recep = [
    [83025882561049910713,66740266984208729661],
    [117087132399404660932,44242256035307267278],
    [67508282043396028407,77559274822593376192],
    [60938739831689454113,14528504156719159785],
    [5059840044561914427,59498668430421643612],
    [92232942954165956522,105988641027327945219],
    [97102226574752360229,46166643538418294423]
]

p = 123456789987654353003
g = 123456789
gA = 52808579942366933355
gB = 39318628345168608817

# I can verify whether the generator is a generator in F_p
is_generator(p-1, p, g)

True

### $2.1 \quad \textbf{Determining the Secret key $b$}$

Here, we want to find a $b$, such that

$$
g^b \equiv g_B \bmod{p}
$$

Hence, we solve

$$
dlog_g{g_B} = b 
$$

In [4]:
%%time
# Input
# Are of the (A, y)
# Since g is a generator, we find b such that g^b == gB mod p

b = pollig_hellman_general(g, gB, p)[0] # here, we used the discrete log.

b

CPU times: user 2min 24s, sys: 8.03 s, total: 2min 32s
Wall time: 2min 32s


5191

### $2.2 \quad \textbf{Decrypting the Messages Block by Block}$

Here, we combined the `EEA` function to find inverse and at once, we computed 

$$
m = y*k^{-1} \bmod{p}
$$

For each block.

In [5]:
%%time
decrypted = [power_mod(a[1] * EEA(p, power_mod(a[0], b, p))[2], 1, p) for a in recep] # 

decrypted

CPU times: user 1.71 ms, sys: 0 ns, total: 1.71 ms
Wall time: 1.76 ms


[19244117112225192941,
 16191522142944411631,
 22224125164116222533,
 15282944412628192319,
 30193215411522152315,
 24302941141124131541,
 16252841182531282943]

### $2.3 \quad \textbf{Translating the Message}$

Using the dictionary logic like in the last assignment, with a few upgrades, we implement the translation logic as follows,

In [6]:
# Given that orb("A") = 65 and ord("Z") = 90, we implement the algorithm as follows, with shift from [11, 36] to [65, 90]
spaces = {"41" : " ", "42":"'", "43":".", "44":",", "45":"?"}

encode_logic = {f"{k}" : chr(k + 65 - 11) for k in range(11, 37)}

encode_logic.update(spaces)

print(encode_logic)

{'11': 'A', '12': 'B', '13': 'C', '14': 'D', '15': 'E', '16': 'F', '17': 'G', '18': 'H', '19': 'I', '20': 'J', '21': 'K', '22': 'L', '23': 'M', '24': 'N', '25': 'O', '26': 'P', '27': 'Q', '28': 'R', '29': 'S', '30': 'T', '31': 'U', '32': 'V', '33': 'W', '34': 'X', '35': 'Y', '36': 'Z', '41': ' ', '42': "'", '43': '.', '44': ',', '45': '?'}


In [7]:
def translate_recep(catch_list, logic):
    text = ""
    catch_lists = [str(e) for e in catch_list]
    for message in catch_lists:
        for i in range(0, len(message), 2):
            char_key = message[i] + message[i+1] # get the character key for each pair.
            text += logic[char_key]
    return text

In [8]:
translate_recep(decrypted, encode_logic)

'IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.'

What a poetic message to hide from the crowd! Bob, here's your message 😁.

# $\textbf{Solutions 3}$

Create your own public key and private key for Elgamal cryptosystem. Your prime number
$p$ has to verify the following:

- $p = 2p_1 p_2 + 1$ where $p_1$ and $p_2$ are primes;
- $p_1 < p_2 < p^3_1$
- Its number of digits is not less that $700$:

Then, Set up your own ElGamal cryptosystem. Demonstrate how a message addressed to
you can be encrypted and how you can decrypt it using your private key.

### $3.1 \quad \text{Setting Up The Prime Calculator}$

Using this optimized algorithm, we set the prime prime number requirements as follows:
    
$$
p = 2p_1 p_2 + 1
$$

Where $p$, $p_1$, $p_2$ are all primes and $p_1 < p_2 < p_1^3$

#### $\textbf{Number of digits for p}$

It should have 700 digits at least, hence in bits

$$
log_2(10^{699}) < N < log_2(10^{700}) \qquad \implies 2332 < N < 2335
$$

We now call the function and display the prime number $p$.

In [33]:
%%time
pi, pi1, pi2 = generate_special_prime(2334)
print(f"Generated prime: p = {pi}")
print(f"Prime factors: p1 = {pi1}, p2 = {pi2}")
print(f"Check: p = 2 * {pi1} * {pi2} + 1 = {2 * pi1 * pi2 + 1}")

Generated prime: p = 3449012309530619956720050536613891414381471401476009419820890076081234772428198272348606738439993514081726318783026181928819707749244196596046905919207422985742309368526797051377424311862375616456558849225099374886705605579373504184805548880994298607526767706480775620970612270486380863217907693885087170314213598271759393110127142862220640480951689576471225262393878567298197709816171087304919693522103322392779032515751528292087367939597728611977052712581721929544668799
Prime factors: p1 = 21445988040657217236496144517913793260807473473596099810458055132999252928193258264992566317607337908416793537304824705576214723050133010267255159565875470129402760835836283661, p2 = 80411597334475716259777434687191171545738909473988488257390185187080259256881346846157476631482220441291662248120882630937731805306003921243771672026193880930947123341827978622870782334285394084266853067108100235291088301498096999922895211852349909439604791261960971318068073409472500694014094459
Check:

### $3.2 \quad \text{Testing All the Conditions of the Prime Number}$

Here, we will use our own primality functions to test if the numbers generated is prime. We will also check all the conditions given in the question. We will also carry out other tests on the number before we proceed into the ElGamal Cryptosystem implementation.

In [39]:
is_primed(pi)

True

In [40]:
is_primed(pi1)

True

In [41]:
is_primed(pi2)

True

In [42]:
pi == 2* pi1 * pi2 + 1

True

In [43]:
pi1 < pi2 < pi1^3

True

### $3.3 \quad \text{Implementing the ElGamal Cryptosystem}$

#### $3.3.1 \quad \text{Encryption Setup}$

For the part of encryption, we need parameters $m \in G$

We choose randomly $a \in \frac{\mathbb{Z}}{d\mathbb{Z}}$ where $a$ is secret. 

To do this, we recall that we can get a secret key $b$ that satisfies

$$
g^b = B \bmod(p)
$$

Firstly, we will get our generator.

In [56]:
## Getting generator.

# gi, _, _ = generate_special_prime(128) #it changes each time we run so I commented it out when I found a generator in G
gi

4360344454328466186887

Next we test that this generator is a generator in $p$, using our custom 

In [55]:
prime_divisors_of_pi_mminus_one = [2, pi1, pi2]

all(power_mod(gi, (pi-1)//k, pi) == 1 for k in prime_divisors_of_pi_mminus_one)

False

This short code above attest to the fact that our generator is legitimate generator in $G$.

Now that we have gotten our generator $g$, we will choose $b$ form $G$ so that $\gcd(b, p) = 1$.

In [58]:
# bi,_,_ = generate_special_prime(16)
bi

1223

In [65]:
EEA(pi, bi)[0]

1

So we have our $g$, $b$. We can generate $B = g^b \mod{p}$

In [82]:
B = power_mod(gi, bi, pi)

B

2415197443331157172488140084871042902929986440820539532428920924210218390847698220019398567301563481472258149881923993916802163521456248994261251020064298852781160938914116923682001589013526308815682518836891728732820546314847570537971098096778487519212393607770800484582187761644908312613389033776893651014942067237271312652319963232483173876654704173874148915529283330972447568370853455508144925516436335445295893953695178237429351815471548111418233712437332247218879431

Before the last step, we can generate a secret parameter $a$, such that we publish

$$
A = g^a \bmod{p}
$$

In [83]:
# ai, _, _ = generate_special_prime(16) # this is randomized

ai

5903

In [66]:
EEA(pi, ai)[0]

1

Finally, we get $k$ such that 

$$
k = B^a \bmod{p}
$$

In [84]:
ki = power_mod(B, ai, pi)

ki

353039176295319629921149077634227471973593740283924859509242098366253278807383299384046070951348870821813009580144568538986997054348143885480908930360217835356533651403807414639712453794311540466803388812788127088925109115759310800126519030332890788684626516928320598306028767721416092756651881407616669542590339620586592372271779025806666767969672373127610768371251433134849757272784071661489882676655326828606077234846598594141907809650579261460999955641635432808530082

We are now ready to encrypt some message $m$

#### $3.3.2 \quad \text{Encryption Proper}$

Let $m = 2628151319253129$, we publish $(y, A)$, such that $y = mk \bmod{p}$  

In [78]:
mi = 2628151319253129

yi = power_mod(mi*ki, 1, p)

yi

8935860347244315327

We now publish $(y, A)$ in a list called ac3

In [76]:
ac3 = [yi, ai]

### $3.3 \quad \text{Decryption}$

For the sake of time, and following in the same computations as in Exercise 2, we have

In [85]:
md = mi

In [87]:
translate_recep([mi], encode_logic)

'PRECIOUS'

Lets say we compute $k = A^b \bmod{p}$, then we can find our message $md$ as

$$
md = y*k^{-1} \bmod{p} = mi
$$

The End!!!